##**Restaurant Turnover Prediction**

###**Business Context**

The first thing any visitor to India will take in — probably while staring out the window in awe as their aeroplane descends is the sheer size of this country. It is densely populated and patchworked with distinct neighbourhoods, each with its own culinary identity. It would take several lifetimes to get to know all of the street stands, holes in the wall, neighbourhood favourites, and high-end destinations in this city.
And for Indians dining out is and always will be a joyous occasion. Everyone has their own favourite restaurants in the city starting from the street food stall across the street to the 5-star restaurants in the heart of the city. Some are favourites because of the memory attached to it and some are favourites because of the fact that the place has a fantastic ambience. There are a lot of other factors as well which contribute to the likeness of the restaurants which in turn determines their popularity among the masses.

If you look at this from the business perspective for a restaurant, more popularity may mean more visits to the joint increasing the annual turnover of the restaurants. For any restaurant to survive and do well, the annual turnover of the restaurants has to be substantial.

This problem takes a shot at predicting the annual turnover of a set of restaurants across India based on a set of variables given in the data set. This includes the data related to the restaurant such as location, opening date, cuisine type, themes etc. This also includes data pooled from different sources such as social media popularity index, Zomato ratings, etc. Lastly, it also adds a different flavour to the problem by looking at the Customer survey data as well as ratings provided by mystery visitor data (audit done by a third party).

###**Objective**
The goal of this problem is to predict the Annual Turnover of a restaurant based on the variables provided in the data set.

###**Metric to measure**

The measure of accuracy will be RMSE (Root mean square error)

The predicted Annual Turnover for each restaurant in the Test dataset will be compared with the actual Annual Turnover to calculate the RMSE value of the entire prediction. The lower the RMSE value, the better the model will be.

In [8]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Install necessary packages
!pip install optuna catboost xgboost scikit-learn category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 17.2 MB/s eta 0:00:00


In [3]:
#update xgboost
!pip install --upgrade xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 5.2 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4


In [4]:
#import necessary libraries and modules
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from datetime import datetime
import re
from collections import Counter
import optuna
# Import additional models
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor # Another alternative GBDT
from sklearn.ensemble import StackingRegressor
import category_encoders as ce

In [5]:
#load the datasets
train_df_orig = pd.read_csv('/content/drive/MyDrive/UT AI ML PG program/Hackathon1/Day 1/Train_dataset.csv')
test_df_orig = pd.read_csv('/content/drive/MyDrive/UT AI ML PG program/Hackathon1/Day 1/Test_dataset.csv')

In [6]:
#model building
test_df_orig.rename(columns={'Endoresed By': 'Endorsed By'}, inplace=True)

train_reg_nums = train_df_orig['Registration Number']
test_reg_nums = test_df_orig['Registration Number']
train_target = train_df_orig['Annual Turnover']
train_df = train_df_orig.drop('Annual Turnover', axis=1)
test_df = test_df_orig.copy()

combined_df = pd.concat([train_df, test_df], ignore_index=True, sort=False)
train_len = len(train_df)

# --- Feature Engineering & Preprocessing (Same as before) ---
# 1. Opening Day of Restaurant
combined_df['Opening Day of Restaurant'] = pd.to_datetime(combined_df['Opening Day of Restaurant'], format='%d-%m-%Y', errors='coerce')
latest_date_in_data = combined_df['Opening Day of Restaurant'].max()
reference_date = datetime(2014, 3, 1)
if latest_date_in_data is not pd.NaT and latest_date_in_data > reference_date :
    reference_date = latest_date_in_data + pd.Timedelta(days=1)

combined_df['Restaurant_Age_Days'] = (reference_date - combined_df['Opening Day of Restaurant']).dt.days
combined_df['Opening_Year'] = combined_df['Opening Day of Restaurant'].dt.year
combined_df['Opening_Month'] = combined_df['Opening Day of Restaurant'].dt.month
combined_df['Opening_DayOfWeek'] = combined_df['Opening Day of Restaurant'].dt.dayofweek

date_cols_to_impute = ['Restaurant_Age_Days', 'Opening_Year', 'Opening_Month', 'Opening_DayOfWeek']
for col in date_cols_to_impute:
    median_val_train = combined_df.loc[:train_len-1, col].median()
    combined_df[col].fillna(median_val_train, inplace=True)
combined_df = combined_df.drop('Opening Day of Restaurant', axis=1)

# 2. Numerical Columns
numerical_cols_to_impute_median = ['Facebook Popularity Quotient', 'Instagram Popularity Quotient']
for col in numerical_cols_to_impute_median:
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
    median_val_train = combined_df.loc[:train_len-1, col].median()
    combined_df[col].fillna(median_val_train, inplace=True)

rating_cols = ['Restaurant Zomato Rating', 'Order Wait Time', 'Staff Responsivness', 'Value for Money',
               'Hygiene Rating', 'Food Rating', 'Overall Restaurant Rating', 'Live Music Rating',
               'Comedy Gigs Rating', 'Value Deals Rating', 'Live Sports Rating', 'Ambience',
               'Lively', 'Service', 'Comfortablility', 'Privacy']
for col in rating_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].replace('NA', np.nan)
        combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')
        combined_df[col].fillna(-1, inplace=True)

# 3. Categorical Columns
# City
combined_df['City'] = combined_df['City'].astype(str).str.lower().str.strip()
combined_df['City'] = combined_df['City'].replace(['-1', 'nan', '', 'na'], 'unknown_city')
city_counts_train = combined_df.loc[:train_len-1, 'City'].value_counts()
# Increased threshold slightly for city grouping
rare_cities_train = city_counts_train[city_counts_train < 15].index
combined_df['City'] = combined_df['City'].apply(lambda x: 'other_city' if x in rare_cities_train and x != 'unknown_city' else x)

# Cuisine
combined_df['Cuisine'] = combined_df['Cuisine'].fillna('unknown').astype(str).str.lower()
cuisine_list_series = combined_df['Cuisine'].apply(lambda x: [c.strip() for c in x.split(',') if c.strip()])
combined_df['Num_Cuisines'] = cuisine_list_series.apply(len)
flat_cuisine_list_train = [item for sublist in cuisine_list_series[:train_len] for item in sublist if item != 'unknown']
cuisine_counts_train_dict = Counter(flat_cuisine_list_train)
# Slightly more cuisines
top_n_cuisines = [cuisine for cuisine, count in cuisine_counts_train_dict.most_common(30)]
for cuisine_name in top_n_cuisines:
    cleaned_name = re.sub(r'\W+', '_', cuisine_name)
    combined_df[f'Cuisine_{cleaned_name}'] = cuisine_list_series.apply(lambda x: 1 if cuisine_name in x else 0)
combined_df = combined_df.drop('Cuisine', axis=1)

# Restaurant Theme
combined_df['Restaurant Theme'] = combined_df['Restaurant Theme'].fillna('Unknown_Theme').astype(str)
theme_counts_train = combined_df.loc[:train_len-1, 'Restaurant Theme'].value_counts()
# Slightly more aggressive grouping for themes
rare_themes_train = theme_counts_train[theme_counts_train < 15].index
combined_df['Restaurant Theme'] = combined_df['Restaurant Theme'].apply(lambda x: 'Other_Theme' if x in rare_themes_train and x != 'Unknown_Theme' else x)

# Resturant Tier
mode_tier_train = combined_df.loc[:train_len-1, 'Resturant Tier'].mode()
fill_tier_value = mode_tier_train[0] if not mode_tier_train.empty else '2.0' # Default if mode is empty
combined_df['Resturant Tier'] = combined_df['Resturant Tier'].fillna(fill_tier_value)
combined_df['Resturant Tier'] = combined_df['Resturant Tier'].astype(str)


cat_cols_simple_impute_mode = ['Restaurant Location', 'Endorsed By', 'Restaurant Type', 'Restaurant City Tier']
for col in cat_cols_simple_impute_mode:
    mode_val_train = combined_df.loc[:train_len-1, col].mode()
    fill_value = mode_val_train[0] if not mode_val_train.empty else 'Unknown'
    combined_df[col].fillna(fill_value, inplace=True)
    combined_df[col] = combined_df[col].astype(str)

binary_cols = ['Fire Audit', 'Liquor License Obtained', 'Situated in a Multi Complex', 'Dedicated Parking', 'Open Sitting Available']
for col in binary_cols:
    if col in combined_df.columns:
        combined_df[col] = combined_df[col].astype(int)
combined_df = combined_df.drop('Registration Number', axis=1)

categorical_features_for_ohe = combined_df.select_dtypes(include=['object']).columns.tolist()
if 'Resturant Tier' not in categorical_features_for_ohe: categorical_features_for_ohe.append('Resturant Tier')
if 'Restaurant City Tier' not in categorical_features_for_ohe: categorical_features_for_ohe.append('Restaurant City Tier')
categorical_features_for_ohe = [col for col in categorical_features_for_ohe if not col.startswith('Cuisine_')]
combined_df = pd.get_dummies(combined_df, columns=[col for col in categorical_features_for_ohe if col in combined_df.columns], dummy_na=False)

processed_train_df = combined_df.iloc[:train_len].copy()
processed_test_df = combined_df.iloc[train_len:].copy()
train_cols = set(processed_train_df.columns)
test_cols = set(processed_test_df.columns)
for col in train_cols:
    if col not in test_cols:
        processed_test_df[col] = 0
for col in test_cols:
    if col not in train_cols:
        processed_train_df[col] = 0
common_columns_ordered = processed_train_df.columns.tolist()
processed_test_df = processed_test_df[common_columns_ordered]

original_numerical_continuous_cols = [
    'Facebook Popularity Quotient', 'Instagram Popularity Quotient',
    'Restaurant Zomato Rating', 'Order Wait Time', 'Staff Responsivness',
    'Value for Money', 'Hygiene Rating', 'Food Rating', 'Overall Restaurant Rating',
    'Live Music Rating', 'Comedy Gigs Rating', 'Value Deals Rating', 'Live Sports Rating',
    'Ambience', 'Lively', 'Service', 'Comfortablility', 'Privacy',
    'Restaurant_Age_Days', 'Opening_Year', 'Opening_Month', 'Opening_DayOfWeek',
    'Num_Cuisines']
numerical_features_to_scale = [col for col in original_numerical_continuous_cols if col in processed_train_df.columns]

if numerical_features_to_scale:
    scaler = StandardScaler()
    processed_train_df[numerical_features_to_scale] = scaler.fit_transform(processed_train_df[numerical_features_to_scale])
    processed_test_df[numerical_features_to_scale] = scaler.transform(processed_test_df[numerical_features_to_scale])

train_target_log = np.log1p(train_target)

# --- Optuna Objective Function ---
def objective(trial):
    # Define hyperparameter search space
    params = {
        'objective': 'regression_l1', # Changed to L1 for potentially sparser solutions
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000), # Wider range for n_estimators
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05), # Slightly lower max LR
        'num_leaves': trial.suggest_int('num_leaves', 20, 100), # Wider range for num_leaves
        'max_depth': trial.suggest_int('max_depth', 5, 15), # Explicit max_depth
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 1e-8, 1.0, log=True),
        'verbose': -1,
        'n_jobs': -1,
        'seed': 42,
        'boosting_type': 'gbdt',
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=123) # Using 5 splits for faster Optuna trials
    oof_rmse_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(processed_train_df, train_target_log)):
        X_train, y_train = processed_train_df.iloc[train_idx], train_target_log.iloc[train_idx]
        X_val, y_val = processed_train_df.iloc[val_idx], train_target_log.iloc[val_idx]

        model = lgb.LGBMRegressor(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  eval_metric='rmse',
                  callbacks=[lgb.early_stopping(100, verbose=False)]) # Reduced patience for early stopping

        preds_val = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds_val))
        oof_rmse_scores.append(rmse)

    return np.mean(oof_rmse_scores)

# --- Run Optuna Study ---
study = optuna.create_study(direction='minimize', study_name='lgbm_turnover_regression')
# Number of trials - can be increased for more thorough search
study.optimize(objective, n_trials=60, timeout=1200) # 60 trials or 20 minutes timeout

best_params = study.best_params
print("Best hyperparameters found by Optuna:", best_params)

# --- Model Training with Best Parameters ---
# Using K-Fold for final prediction averaging, as it often yields more robust results
final_params_lgb = {
    'objective': 'regression_l1',
    'metric': 'rmse',
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}
final_params_lgb.update(best_params) # Update with Optuna's best params

kf_final = KFold(n_splits=10, shuffle=True, random_state=42) # Back to 10 splits for final model
oof_predictions_final = np.zeros(len(processed_train_df))
test_predictions_final = np.zeros(len(processed_test_df))

for fold, (train_idx, val_idx) in enumerate(kf_final.split(processed_train_df, train_target_log)):
    X_train, y_train = processed_train_df.iloc[train_idx], train_target_log.iloc[train_idx]
    X_val, y_val = processed_train_df.iloc[val_idx], train_target_log.iloc[val_idx]

    model = lgb.LGBMRegressor(**final_params_lgb)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],
              eval_metric='rmse',
              callbacks=[lgb.early_stopping(150, verbose=False)]) # Longer patience for final model

    oof_predictions_final[val_idx] = model.predict(X_val)
    test_predictions_final += model.predict(processed_test_df) / kf_final.n_splits

# --- Evaluation and Submission ---
oof_rmse_log_final = np.sqrt(mean_squared_error(train_target_log, oof_predictions_final))
print(f"\nFinal OOF RMSE (log-transformed target) with best params: {oof_rmse_log_final}")

oof_predictions_orig_scale_final = np.expm1(oof_predictions_final)
oof_predictions_orig_scale_final[oof_predictions_orig_scale_final < 0] = 0
oof_rmse_orig_final = np.sqrt(mean_squared_error(train_target, oof_predictions_orig_scale_final))
print(f"Final OOF RMSE (original scale) with best params: {oof_rmse_orig_final}")

final_predictions_log_tuned = test_predictions_final
final_predictions_tuned = np.expm1(final_predictions_log_tuned)
final_predictions_tuned[final_predictions_tuned < 0] = 0

submission_df_tuned = pd.DataFrame({
    'Registration Number': test_reg_nums,
    'Annual Turnover': final_predictions_tuned
})

submission_df_tuned.to_csv("submission_tuned.csv", index=False)
print("Tuned submission file created: submission_tuned.csv")

if oof_rmse_orig_final < 12200000:
    print(f"\nSuccess! The OOF RMSE ({oof_rmse_orig_final:.2f}) is below 12,200,000.")
else:
    print(f"\nFurther tuning or feature engineering might be needed. Current OOF RMSE: {oof_rmse_orig_final:.2f}")

<ipython-input-6-98c1ecf2e960>:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df[col].fillna(median_val_train, inplace=True)
<ipython-input-6-98c1ecf2e960>:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

Best hyperparameters found by Optuna: {'n_estimators': 4175, 'learning_rate': 0.0071555998771090455, 'num_leaves': 22, 'max_depth': 10, 'min_child_samples': 8, 'feature_fraction': 0.5471487479142861, 'bagging_fraction': 0.5614118994206465, 'bagging_freq': 6, 'lambda_l1': 0.01667949466079257, 'lambda_l2': 0.012432291348663504, 'min_split_gain': 0.013168564451584682}

Final OOF RMSE (log-transformed target) with best params: 0.46693251755879744
Final OOF RMSE (original scale) with best params: 20137812.374023587
Tuned submission file created: submission_tuned.csv

Further tuning or feature engineering might be needed. Current OOF RMSE: 20137812.37


In [7]:
# --- XGBoost Model (Added) ---
# Optuna Objective for XGBoost
def objective_xgb(trial):
    params = {
        'objective': 'reg:squarederror', # Use RMSE objective
        'eval_metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True), # L2 regularization
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),  # L1 regularization
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),     # Minimum loss reduction required to make a split
        'seed': 42,
        'n_jobs': -1,
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=123)
    oof_rmse_scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(processed_train_df, train_target_log)):
        X_train, y_train = processed_train_df.iloc[train_idx], train_target_log.iloc[train_idx]
        X_val, y_val = processed_train_df.iloc[val_idx], train_target_log.iloc[val_idx]

        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  verbose=False
                 )

        preds_val = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds_val))
        oof_rmse_scores.append(rmse)

    return np.mean(oof_rmse_scores)

print("\nStarting Optuna study for XGBoost...")
study_xgb = optuna.create_study(direction='minimize', study_name='xgb_turnover_regression')
study_xgb.optimize(objective_xgb, n_trials=60, timeout=1200) # Run 60 trials or for 20 minutes

best_params_xgb = study_xgb.best_params
print("Best hyperparameters found by Optuna for XGBoost:", best_params_xgb)

# --- CatBoost Model (Added) ---
# Optuna Objective for CatBoost
def objective_cat(trial):
    params = {
        'objective': 'RMSE',
        'iterations': trial.suggest_int('iterations', 1000, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05),
        'depth': trial.suggest_int('depth', 5, 10), # CatBoost depth typically lower
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'random_seed': 42,
        'verbose': 0, # Suppress verbose output during trials
        'early_stopping_rounds': trial.suggest_int('early_stopping_rounds', 50, 200),
        # Removed 'Poisson' as it's not supported on CPU
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS', 'No']),
    }

    # Conditionally suggest bagging_temperature and subsample based on bootstrap_type
    bootstrap_type = trial.params['bootstrap_type']
    if bootstrap_type == 'Bayesian':
         params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 1)
    elif bootstrap_type in ['Bernoulli', 'MVS']:
         params['subsample'] = trial.suggest_float('subsample', 0.5, 0.9)


    kf = KFold(n_splits=5, shuffle=True, random_state=123)
    oof_rmse_scores = []

    # CatBoost requires specifying categorical features explicitly
    # Need to reverse one-hot encoding to find original categorical columns for CatBoost
    # Note: This requires knowing the original column names before OHE
    # For simplicity here, let's assume no explicit categorical feature handling in CatBoost for now,
    # as it works well with numeric features. If performance is poor, this is a place to optimize.
    # CatBoost can handle integer-encoded categorical features directly. Our OHE prevents this.
    # If we wanted to use CatBoost's categorical feature handling, we'd need to use a different preprocessing path for it.
    # For now, we'll treat all features the same as for LGBM and XGBoost.

    for fold, (train_idx, val_idx) in enumerate(kf.split(processed_train_df, train_target_log)):
        X_train, y_train = processed_train_df.iloc[train_idx], train_target_log.iloc[train_idx]
        X_val, y_val = processed_train_df.iloc[val_idx], train_target_log.iloc[val_idx]

        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=params['early_stopping_rounds'],
                  verbose=0)

        preds_val = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, preds_val))
        oof_rmse_scores.append(rmse)

    return np.mean(oof_rmse_scores)

print("\nStarting Optuna study for CatBoost...")
study_cat = optuna.create_study(direction='minimize', study_name='cat_turnover_regression')
study_cat.optimize(objective_cat, n_trials=60, timeout=1200) # Run 60 trials or for 20 minutes

best_params_cat = study_cat.best_params
print("Best hyperparameters found by Optuna for CatBoost:", best_params_cat)


# --- Final Model Training and Averaging ---
kf_ensemble = KFold(n_splits=10, shuffle=True, random_state=42)

oof_preds_lgbm = np.zeros(len(processed_train_df))
test_preds_lgbm = np.zeros(len(processed_test_df))
oof_preds_xgb = np.zeros(len(processed_train_df))
test_preds_xgb = np.zeros(len(processed_test_df))
oof_preds_cat = np.zeros(len(processed_train_df))
test_preds_cat = np.zeros(len(processed_test_df))

lgbm_oof_rmses = []
xgb_oof_rmses = []
cat_oof_rmses = []

print("\nTraining base models and collecting OOF and Test predictions...")
for fold, (train_idx, val_idx) in enumerate(kf_ensemble.split(processed_train_df, train_target_log)):
    print(f"--- Fold {fold+1} ---")
    X_train, y_train = processed_train_df.iloc[train_idx], train_target_log.iloc[train_idx]
    X_val, y_val = processed_train_df.iloc[val_idx], train_target_log.iloc[val_idx]
    X_test = processed_test_df.copy() # Use the entire test set for each fold's test prediction

    # LightGBM
    print("  Training LightGBM...")
    lgbm_model = lgb.LGBMRegressor(**final_params_lgb) # Use best params from initial Optuna
    lgbm_model.fit(X_train, y_train,
                   eval_set=[(X_val, y_val)],
                   eval_metric='rmse',
                   callbacks=[lgb.early_stopping(150, verbose=False)])
    oof_preds_lgbm[val_idx] = lgbm_model.predict(X_val)
    test_preds_lgbm += lgbm_model.predict(X_test) / kf_ensemble.n_splits

    # XGBoost
    print("  Training XGBoost...")
    xgb_model = xgb.XGBRegressor(**best_params_xgb) # Use best params from XGBoost Optuna
    xgb_model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  verbose=False,
                 )
    oof_preds_xgb[val_idx] = xgb_model.predict(X_val)
    test_preds_xgb += xgb_model.predict(X_test) / kf_ensemble.n_splits

    # CatBoost
    print("  Training CatBoost...")
    cat_model = CatBoostRegressor(**best_params_cat) # Use best params from CatBoost Optuna
    # Need to handle categorical features for CatBoost if not using OHE, but we used OHE.
    # So, fit on processed_train_df which has OHE features.
    cat_model.fit(X_train, y_train,
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=best_params_cat['early_stopping_rounds'],
                  verbose=0)
    oof_preds_cat[val_idx] = cat_model.predict(X_val)
    test_preds_cat += cat_model.predict(X_test) / kf_ensemble.n_splits


# Calculate OOF RMSE for each model (on original scale)
oof_preds_lgbm_orig = np.expm1(oof_preds_lgbm)
oof_preds_lgbm_orig[oof_preds_lgbm_orig < 0] = 0
rmse_lgbm_oof = np.sqrt(mean_squared_error(train_target, oof_preds_lgbm_orig))
print(f"\nLightGBM OOF RMSE (original scale): {rmse_lgbm_oof:.2f}")

oof_preds_xgb_orig = np.expm1(oof_preds_xgb)
oof_preds_xgb_orig[oof_preds_xgb_orig < 0] = 0
# Corrected variable name from 'of_preds_xgb_orig' to 'oof_preds_xgb_orig'
rmse_xgb_oof = np.sqrt(mean_squared_error(train_target, oof_preds_xgb_orig))
print(f"XGBoost OOF RMSE (original scale): {rmse_xgb_oof:.2f}")

oof_preds_cat_orig = np.expm1(oof_preds_cat)
oof_preds_cat_orig[oof_preds_cat_orig < 0] = 0
rmse_cat_oof = np.sqrt(mean_squared_error(train_target, oof_preds_cat_orig))
print(f"CatBoost OOF RMSE (original scale): {rmse_cat_oof:.2f}")

# Calculate weights based on inverse of OOF RMSE
# Add a small epsilon to avoid division by zero if RMSE is extremely close to zero
epsilon = 1e-6
weight_lgbm = 1 / (rmse_lgbm_oof + epsilon)
weight_xgb = 1 / (rmse_xgb_oof + epsilon)
weight_cat = 1 / (rmse_cat_oof + epsilon)

total_weight = weight_lgbm + weight_xgb + weight_cat

normalized_weight_lgbm = weight_lgbm / total_weight
normalized_weight_xgb = weight_xgb / total_weight
normalized_weight_cat = weight_cat / total_weight

print(f"\nWeights for ensemble:")
print(f"  LightGBM: {normalized_weight_lgbm:.4f}")
print(f"  XGBoost:  {normalized_weight_xgb:.4f}")
print(f"  CatBoost: {normalized_weight_cat:.4f}")

# Combine test predictions using weighted average (on log scale, then expm1)
ensemble_test_predictions_log = (normalized_weight_lgbm * test_preds_lgbm +
                                normalized_weight_xgb * test_preds_xgb +
                                normalized_weight_cat * test_preds_cat)

# Convert back to original scale
ensemble_test_predictions_orig = np.expm1(ensemble_test_predictions_log)
ensemble_test_predictions_orig[ensemble_test_predictions_orig < 0] = 0 # Ensure no negative predictions

# --- Create Submission File ---
submission_df_ensemble = pd.DataFrame({
    'Registration Number': test_reg_nums,
    'Annual Turnover': ensemble_test_predictions_orig
})

submission_df_ensemble.to_csv("submission_ensemble.csv", index=False)
print("\nEnsemble submission file created: submission_ensemble.csv")

# Evaluate ensemble OOF RMSE (optional, but good for confidence)
ensemble_oof_predictions_log = (normalized_weight_lgbm * oof_preds_lgbm +
                               normalized_weight_xgb * oof_preds_xgb +
                               normalized_weight_cat * oof_preds_cat)
ensemble_oof_predictions_orig = np.expm1(ensemble_oof_predictions_log)
ensemble_oof_predictions_orig[ensemble_oof_predictions_orig < 0] = 0
ensemble_oof_rmse_orig = np.sqrt(mean_squared_error(train_target, ensemble_oof_predictions_orig))

print(f"\nEnsemble OOF RMSE (original scale): {ensemble_oof_rmse_orig:.2f}")

if ensemble_oof_rmse_orig < 12200000:
    print(f"\nSuccess! The Ensemble OOF RMSE ({ensemble_oof_rmse_orig:.2f}) is below 12,200,000.")
else:
    print(f"\nFurther tuning, feature engineering, or different models might be needed. Current Ensemble OOF RMSE: {ensemble_oof_rmse_orig:.2f}")

[I 2025-06-08 23:08:15,129] A new study created in memory with name: xgb_turnover_regression



Starting Optuna study for XGBoost...


[I 2025-06-08 23:09:39,287] Trial 0 finished with value: 0.4773475260318091 and parameters: {'n_estimators': 4252, 'learning_rate': 0.030806574522629707, 'max_depth': 11, 'min_child_weight': 2, 'subsample': 0.8804380332862003, 'colsample_bytree': 0.7731078308007346, 'lambda': 0.003830588863888105, 'alpha': 4.477770836096484e-07, 'gamma': 0.03550707990395842}. Best is trial 0 with value: 0.4773475260318091.
[I 2025-06-08 23:10:20,755] Trial 1 finished with value: 0.48960276702979366 and parameters: {'n_estimators': 1310, 'learning_rate': 0.049123199998764, 'max_depth': 10, 'min_child_weight': 9, 'subsample': 0.5158745036483404, 'colsample_bytree': 0.7145644806987477, 'lambda': 0.38836990274368566, 'alpha': 0.0009572261514445418, 'gamma': 2.212305670349842e-07}. Best is trial 0 with value: 0.4773475260318091.
[I 2025-06-08 23:12:06,321] Trial 2 finished with value: 0.4911988037571239 and parameters: {'n_estimators': 4159, 'learning_rate': 0.03382375119073215, 'max_depth': 7, 'min_child_w

Best hyperparameters found by Optuna for XGBoost: {'n_estimators': 2407, 'learning_rate': 0.012757899590413468, 'max_depth': 9, 'min_child_weight': 8, 'subsample': 0.5879202675558062, 'colsample_bytree': 0.5758936540041081, 'lambda': 1.4174812478215432e-08, 'alpha': 0.01736218570178793, 'gamma': 0.9136973898476178}

Starting Optuna study for CatBoost...


[I 2025-06-08 23:29:43,391] Trial 0 finished with value: 0.46632229189142754 and parameters: {'iterations': 3704, 'learning_rate': 0.006576818256757044, 'depth': 6, 'l2_leaf_reg': 1.0344035214377897e-07, 'early_stopping_rounds': 162, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.35796284419040214}. Best is trial 0 with value: 0.46632229189142754.
[I 2025-06-08 23:29:58,573] Trial 1 finished with value: 0.4676332624651785 and parameters: {'iterations': 1610, 'learning_rate': 0.03875450303928996, 'depth': 7, 'l2_leaf_reg': 4.353174559704348e-08, 'early_stopping_rounds': 199, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.15593046873363214}. Best is trial 0 with value: 0.46632229189142754.
[I 2025-06-08 23:30:11,313] Trial 2 finished with value: 0.4669866597935444 and parameters: {'iterations': 2195, 'learning_rate': 0.018396326235043368, 'depth': 5, 'l2_leaf_reg': 0.01908886095782474, 'early_stopping_rounds': 73, 'bootstrap_type': 'MVS', 'subsample': 0.8784079719278406}.

Best hyperparameters found by Optuna for CatBoost: {'iterations': 4658, 'learning_rate': 0.010548852408610771, 'depth': 6, 'l2_leaf_reg': 1.3391534905005663e-05, 'early_stopping_rounds': 119, 'bootstrap_type': 'Bernoulli', 'subsample': 0.5854057096788899}

Training base models and collecting OOF and Test predictions...
--- Fold 1 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 2 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 3 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 4 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 5 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 6 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 7 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 8 ---
  Training LightGBM...
  Training XGBoost...
  Training CatBoost...
--- Fold 9 ---
